<a href="https://colab.research.google.com/github/rubyvanrooyen/astrokat/blob/colab_helper_interface/astrokat_catalogue2obsfile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AstroKAT catalogue to observation file
The minimum required information for any observation is a list of observation targets specified as one target per line, using comma separated formatting to provide the relevant target information.

For more detail refer to the
[Catalogues to observation files](https://github.com/ska-sa/astrokat/wiki/Catalogues-to-observation-files)
documentation on the
[AstroKAT](https://github.com/ska-sa/astrokat/wiki)
wiki page.


## Installation and setup

In [ ]:
!pip install git+https://github.com/ska-sa/astrokat.git

In [ ]:
!git clone https://github.com/ska-sa/astrokat.git

In [ ]:
!ln -s /content/astrokat/scripts/astrokat-catalogue2obsfile.py catalogue2obsfile.py

## User input
Upload CSV catalogue containing target coordinates for convertion to observation file

In [ ]:
from google.colab import files
uploaded = files.upload()
filename = next(iter(uploaded))
print(filename)

MeerKAT instrument setup for observation

In [ ]:
#@markdown observation instrument product configuration
product =  '' #@param {type:"string"}
# add lists with values allowed -- and add standard imaging modes
# text_and_dropdown = 'value' #@param ["1st option", "2nd option", "3rd option"] {allow-input: true}

#@markdown observation band
band = 'L' #@param ["L", "UHF", "X", "S"]

#@markdown averaging time per dump [sec]
integration_period = 8.0 #@param {type:"number"}

Target observation strategy.    
Tracking a target for imaging or spectral line observations.

In [ ]:
#@markdown observation start LST [HH:MM] (-1 for auto)
lst = -1 #@param {type:"number"}

#@markdown default target track duration [sec]
target_duration = 300.0 #@param {type:"number"}

#@markdown maximum duration of observation [sec] (-1 for auto)
max_duration = -1 #@param {type:"number"}

Calibrator observation strategy.   
Calibrators are identified by tags in their description strings: `bpcal`, `gaincal`, `fluxcal` and `polcal` respectively

In [ ]:
#@markdown primary calibrators are 'bpcal', 'fluxcal' and 'polcal'

#@markdown minimum duration to track primary calibrators [sec].  
primary_cal_duration = 300. #@param {type:"number"}

#@markdown minimum observation interval between primary calibrators [sec]  (-1 for no cadence)
primary_cal_cadence = -1 #@param {type:"number"}


#@markdown primary calibrators are 'gaincal' and 'delaycal'

#@markdown minimum duration to track gain calibrator [sec]
secondary_cal_duration = 60. #@param {type:"number"}

Sources are specified as a catalogue of targets, with optional timing information

In [ ]:
#@markdown filename for output observation file (default only displays output)
outfile = '' #@param {type:"string"}

## Convert CSV to YAML

In [ ]:
import astrokat
import catalogue2obsfile as cat2obs

In [ ]:
# unpack and verify user input
instrument = {}
if product: instrument['product'] = product
instrument['band'] = band
instrument['integration_period'] = integration_period

def eval_none(val_):
    if val_ < 0:
        return None
    else:
        return val_


lst = eval_none(lst)
max_duration = eval_none(max_duration)
primary_cal_cadence = eval_none(primary_cal_cadence)

if not outfile: outfile = None

In [ ]:
# read targets from catalogue file
cat_obj = cat2obs.UnpackCatalogue(filename)
header, catalogue = cat_obj.read_catalogue(target_duration=target_duration,
                                           gaincal_duration=secondary_cal_duration,
                                           bpcal_duration=primary_cal_duration,
                                           bpcal_interval=primary_cal_cadence)

In [ ]:
# create observation configuration output
obs_plan = cat2obs.BuildObservation(catalogue)
obs_plan.configure(instrument=instrument,
                   obs_duration=max_duration,
                   lst=lst)

In [ ]:
# display observation info or write to YAML file
obs_plan.write_yaml(header=header, outfile=outfile)